In [ ]:
import zipfile

zipfile.ZipFile('training_set.zip', 'r').extractall()
zipfile.ZipFile('test_set.zip', 'r').extractall()

In [83]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 강아지와 고양이 이미지 한개씩 plot
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Dog')
img = cv2.imread('training_set/dogs/dog.1.jpg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.subplot(1, 2, 2)
img = cv2.imread('training_set/cats/cat.1.jpg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title('Cat')

In [ ]:
# 이미지를 64x64로 변환하고 색은 유지
def preprocessing(img):
    img = cv2.resize(img, (64, 64))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
def preprocessing_gray(img):
    img = cv2.resize(img, (64, 64))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

In [ ]:
# 강아지와 고양이 이미지 한개씩 plot
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Dog')
plt.imshow(preprocessing(cv2.imread('training_set/dogs/dog.1.jpg')))
plt.subplot(1, 2, 2)
plt.imshow(preprocessing(cv2.imread('training_set/cats/cat.1.jpg')))
plt.title('Cat')

In [ ]:
# 이번에는 grayscale로 변환
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Dog')
plt.imshow(preprocessing_gray(cv2.imread('training_set/dogs/dog.1.jpg')), cmap='gray')
plt.subplot(1, 2, 2)
plt.imshow(preprocessing_gray(cv2.imread('training_set/cats/cat.1.jpg')), cmap='gray')
plt.title('Cat')


In [ ]:
# 이미지 데이터셋을 넘파이 어레이로 불러오기
def load_data():
    train_images = []
    train_labels = []
    test_images = []
    test_labels = []

    # 훈련 데이터
    for filename in os.listdir('training_set/cats'):
        if filename[-4:] != '.jpg':
            continue
        preprocessed_image = preprocessing(cv2.imread('training_set/cats/' + filename))
        train_images.append(preprocessed_image)
        train_labels.append(np.array([1, 0]))

    for filename in os.listdir('training_set/dogs'):
        if filename[-4:] != '.jpg':
            continue
        preprocessed_image = preprocessing(cv2.imread('training_set/dogs/' + filename))
        train_images.append(preprocessed_image)
        train_labels.append(np.array([0, 1]))

    # 테스트 데이터
    for filename in os.listdir('test_set/cats'):
        if filename[-4:] != '.jpg':
            continue
        preprocessed_image = preprocessing(cv2.imread('test_set/cats/' + filename))
        test_images.append(preprocessed_image)
        test_labels.append(np.array([1, 0]))

    for filename in os.listdir('test_set/dogs'):
        if filename[-4:] != '.jpg':
            continue
        preprocessed_image = preprocessing(cv2.imread('test_set/dogs/' + filename))
        test_images.append(preprocessed_image)
        test_labels.append(np.array([0, 1]))

    return np.array(train_images), np.array(train_labels), np.array(test_images), np.array(test_labels)

In [ ]:
train_X, train_Y, test_X, test_Y = load_data()

# 데이터 셔플
shuffle_indices = np.random.permutation(np.arange(len(train_Y)))
train_X = train_X[shuffle_indices]
train_Y = train_Y[shuffle_indices]

In [84]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [85]:
# 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [86]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 6, 6, 128)       

In [ ]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 모델 훈련을 하면서 test 데이터로 검증
history = model.fit(train_X, train_Y, epochs=50, validation_data=(test_X, test_Y), batch_size=32)

In [ ]:
# history를 통해 훈련 과정을 그래프로 그려보기
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.ylim(0.5, 1)
plt.legend()
plt.show()

In [ ]:
# 학습 파라미터 저장
model.save('model_keras.h5')

In [72]:
# 모델 불러오기
from keras.models import load_model
model2 = load_model('model_keras.h5')